In [24]:
import pandas as pd
import numpy as np
import os
from nice_funcs.indicators import CreateRandomPrtf,EWMA,MACD,RSI,NormalizeWindow,MDD
from stable_baselines3 import DDPG ,PPO
from ambiente import TradingEnv


    

def GetIndex(*args):
  indicators = [*args]
  index_init = set(indicators[0].index)
  for ind_ in indicators:
    index_init = index_init & set(ind_.index)
  
  idx_date = min(index_init)
  new_index_indicators = []
  for ind_ in indicators:
    ind_['Cash'] = 0 
    new_index_indicators.append(ind_[idx_date:])
  return new_index_indicators
        


# %%
path_diario = './assets/1d/'
ativos = os.listdir(path_diario)

ativosOHLC = {}
for ativo in ativos:
    ativosOHLC[ativo.replace('.xlsx','')] = \
        pd.read_excel(os.path.join(path_diario,ativo),index_col=0)
    

close_prices = {}
for k in ativosOHLC.keys():
  close_prices[k] = ativosOHLC[k].Close


# %%
df_fechamento = pd.DataFrame(close_prices)
macd = df_fechamento.apply(lambda row: MACD(row)[0]).dropna()
rsi = df_fechamento.apply(lambda row: RSI(row)).dropna()
ewma_diff = df_fechamento.apply(lambda row: EWMA(row,20) - EWMA(row,5)).dropna()
normalized_fech = df_fechamento.apply(lambda row: NormalizeWindow(row)).dropna()

# %%
  

# %%
df_fechamento = pd.DataFrame(close_prices).iloc[-360:]
normalized_fech = df_fechamento.apply(lambda row: NormalizeWindow(row)).dropna()
macd = df_fechamento.apply(lambda row: MACD(row)[0]).dropna()
rsi = df_fechamento.apply(lambda row: RSI(row)).dropna()
ewma_diff = df_fechamento.apply(lambda row: EWMA(row,20) - EWMA(row,5)).dropna()
ddd = df_fechamento.apply(lambda row: MDD(row,window=26)[0]).dropna()
mdd = df_fechamento.apply(lambda row: MDD(row,window=26)[0]).rolling(window=26).min().dropna()
df_fechamento,normalized_fech,macd,rsi,ewma_diff,ddd,mdd =  GetIndex(df_fechamento,normalized_fech, macd, rsi, ewma_diff,ddd,mdd)






env = TradingEnv(df_fechamento,[normalized_fech,macd,rsi,ewma_diff])

In [30]:
model = PPO.load('./Training/Saved Models/trading_2.zip') 
#model = PPO.load('./Training/Logs/best_model.zip')

In [32]:
stats = {
  'ganho_total':[],
  'total_turnos':[],
  'prtf':[],
  'valor':[]}
for _ in range(1):
  obs,_ = env.reset()
  total_ganho = 0
  done = False
  truncated = False
  termination = (done or truncated)
  acoes = []
  rewards = []
  while not termination:
      action,_ = model.predict(obs,deterministic=True)
      obs,reward, done ,truncated,info = env.step(action)
      total_ganho +=reward
      termination = (done or truncated)
      if sum(action) != 0:
         action = action/sum(action)
      else:
         action = env.softmax_normalization(action)
      acoes.append(action)
      rewards.append(reward)
  stats['ganho_total'].append(total_ganho)
  stats['total_turnos'].append(env.step_)
  stats['prtf'].append(acoes)
  stats['valor'].append(rewards)

**************************************************
Retorno 1427.23


In [33]:
x = (pd.Series(stats['valor'][0]).cumsum() +1000).pct_change().describe()

In [34]:
(x['mean']/x['std'])*np.sqrt(360)

0.9658497411869824

In [16]:
(pd.DataFrame(stats['prtf'][0],columns=df_fechamento.columns)*100).mean()

ADAUSDT    10.170621
BNBUSDT    28.995121
BTCUSDT    18.725794
ETHUSDT    12.656740
LTCUSDT     7.255637
XRPUSDT     6.038750
Cash       16.157339
dtype: float32

In [7]:
returns = df_fechamento.pct_change().iloc[1:].fillna(0)

In [8]:
log_returns = np.log(1+returns)

In [9]:
equal_pesos = [1/len(log_returns.columns)]*len(log_returns.columns)

In [10]:
prtf_returns = (log_returns * equal_pesos).sum(axis=1)

In [11]:
(prtf_returns.mean()/prtf_returns.std())*np.sqrt(360)

0.27148376063109264